# 🛍️ Retail Product Recommender System using SnowparkML

## 📋 Overview
This notebook demonstrates an end-to-end machine learning pipeline for building a product recommendation system using Snowpark ML. The system analyzes retail transaction data to generate personalized product recommendations using Bayesian Personalized Ranking.

## 🔄 Pipeline Components
1. **🔍 Data Loading & Preparation**: Extract retail transaction data from Snowflake stage
2. **📊 Data Exploration & Analysis**: Understand customer purchase patterns and product relationships
3. **⚙️ Feature Engineering**: Convert transaction data into user-item interaction matrices
4. **🧠 Model Training**: Implement BPR algorithm with hyperparameter tuning
5. **📏 Model Evaluation**: Calculate precision, recall, and coverage metrics
6. **🚀 Model Deployment**: Register model for real-time recommendations
7. **📈 Performance Visualization**: Interactive dashboards for model insights

## 💼 Business Value
This recommendation system enables retailers to:
- 💰 Increase average order value through personalized product suggestions
- 🤝 Enhance customer experience with relevant recommendations
- 🔄 Identify cross-selling opportunities based on purchase patterns
- 📦 Optimize inventory management based on predicted demand

In [ ]:
"""
Retail Recommender System - Streamlit App
Sets up the environment for product recommendations using Snowflake and Streamlit.
"""

# Core data and visualization libraries
import streamlit as st             # Web app framework for interactive UI
import pandas as pd                # Data manipulation and analysis
import numpy as np                 # Numerical operations
import matplotlib.pyplot as plt    # Base plotting library
import seaborn as sns              # Statistical data visualization
from datetime import datetime, timedelta  # Date handling
import warnings                    # Warning control
import joblib                      # Model persistence
import time                        # Time utilities for performance tracking
import sys


# Snowflake libraries
from snowflake.snowpark.context import get_active_session  # Access current Snowflake connection
from snowflake.snowpark.types import StringType, IntegerType  # Snowflake data types
import snowflake.snowpark.functions as F  # Snowflake SQL functions
from snowflake.ml.registry import Registry  # Model registry for MLOps
from snowflake.ml.modeling.pipeline import Pipeline  # ML pipeline management
import snowflake.ml.modeling.preprocessing as pp  # Snowflake-specific preprocessing
from snowflake.ml.modeling.xgboost import XGBClassifier  # Gradient boosting implementation
from IPython.display import Markdown, display  # Rich output display
from implicit.bpr import BayesianPersonalizedRanking  # Recommendation algorithm
from sklearn.model_selection import train_test_split  # Dataset splitting
from matplotlib.ticker import FuncFormatter  # Custom axis formatting

# Configure pandas display settings
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)        # Set wide display width
pd.set_option('display.max_colwidth', None) # Show full content of each cell
warnings.filterwarnings('ignore')           # Suppress warning messages

# ML and analytics libraries
from scipy.sparse import coo_matrix  # Sparse matrix for recommendation models
from sklearn.preprocessing import LabelEncoder  # Convert categorical variables

def setup_snowflake_session():
    """Configure Snowflake session with tracking tags"""
    session = get_active_session()  # Get current Snowflake session
    # Add metadata tags for tracking and governance
    session.query_tag = {
        "origin": "sf_demo-retail-recommender",
        "name": "mlops_product_recommendation",
        "version": {"major": 1, "minor": 0}
    }
    return session

def configure_environment():
    """Set up visualization and pandas configuration"""
    # Configure pandas and suppress warnings
    pd.set_option("display.max_columns", 500)  # Ensure we see all columns in outputs
    pd.set_option("display.max_rows", 10)      # Limit default row display
    pd.options.mode.chained_assignment = None  # Disable chained assignment warning
    warnings.filterwarnings('ignore', category=FutureWarning)  # Suppress future warnings
    warnings.filterwarnings('ignore', category=UserWarning, message='.*tight_layout.*')  # Suppress layout warnings
    
    # Set visualization style
    plt.style.use('seaborn-v0_8-whitegrid')  # Clean, professional visualization style
    return sns.color_palette("viridis", n_colors=10)  # Return color palette for consistent visuals

def main():
    """Initialize session and environment"""
    session = setup_snowflake_session()  # Configure Snowflake connection
    custom_palette = configure_environment()  # Set up visualization settings
    solution_prefix = session.get_current_warehouse()  # Get warehouse name for reference
    return session, solution_prefix, custom_palette

# Execute setup based on context
if __name__ == "__main__":
    # Direct execution path (when run as a script)
    session, solution_prefix, custom_palette = main()
    st.write(f"Connected to warehouse: {solution_prefix}")  # Display connection info in Streamlit
else:
    # Import path (when imported as a module)
    session = setup_snowflake_session()
    custom_palette = configure_environment()
    solution_prefix = session.get_current_warehouse()


## 🛒 Snowflake Environment Setup

This SQL block configures the necessary Snowflake components for our retail recommender system:

- Creates a custom CSV file format that skips header rows
- Establishes a dedicated storage stage for retail data files
- Includes reference commands for uploading CSV data files

The setup provides a standardized environment for consistent data ingestion and processing throughout our recommendation pipeline.

In [ ]:
-- Set up data format and stage for retail recommender
-- Define CSV format for ingesting retail data
CREATE FILE FORMAT IF NOT EXISTS CSVFORMAT 
    SKIP_HEADER = 1   -- Skip header row in CSV files
    TYPE = 'CSV';     -- Use CSV format for data ingestion
    
-- Create stage to store and access data files
CREATE OR REPLACE STAGE snowflake_recommender_system_with_implicit_bpr_stage
    FILE_FORMAT = (TYPE = 'CSV');

-- Commented options for file upload to stage:
-- PUT file:///online_retail_2010.csv snowflake_recommender_system_with_implicit_bpr_stage;
-- PUT file://online_retail_2010.csv @snowflake_recommender_system_with_implicit_bpr_stage AUTO_COMPRESS=FALSE;


## 📊 Data Preparation and Staging

This code block handles the initial data preparation by:

1. Importing our retail dataset from an Excel file containing multiple sheets
2. Consolidating all worksheets into a single unified DataFrame
3. Converting the consolidated data to CSV format for compatibility with Snowflake
4. Uploading the prepared dataset to our previously configured Snowflake stage

This step bridges the gap between raw source data and our cloud-based analytics environment, making the retail transaction history available for subsequent processing and model building steps.

In [ ]:
# Process Excel data and upload to Snowflake stage
all_sheets = pd.read_excel("online_retail_II.xlsx", sheet_name=None)  # Load all sheets
combined_df = pd.concat(all_sheets.values(), ignore_index=True)       # Merge sheets into one dataframe

# Export to CSV and upload to Snowflake
combined_df.to_csv("online_retail_II.csv", index=False)               # Convert to CSV format
session.file.put('online_retail_II.csv', "@snowflake_recommender_system_with_implicit_bpr_stage", auto_compress=False)  # Upload to stage

## 📂 Stage Content Verification

This simple SQL command lists all files currently stored in our `retail_recommender_stage` storage location. Running this command helps verify that our data upload was successful and allows us to confirm the availability of required datasets before proceeding with data processing and model development.

In [ ]:
LS @snowflake_recommender_system_with_implicit_bpr_stage;

## 📥 Data Loading and Preview

This code block transitions our data from raw files in the Snowflake stage to a structured Snowpark DataFrame for analysis:

1. Reads the CSV data with appropriate configuration for delimiters and encoding
2. Enables schema inference to automatically detect appropriate data types
3. Properly handles the CSV header to establish column names
4. Displays a formatted preview of the first 10 rows to verify data quality

This step establishes our working dataset and allows us to visually confirm the structure and content before moving forward with data cleaning and feature engineering.

In [ ]:
# Load CSV data from stage into Snowpark dataframe
spdf = session.read.options({
    "field_delimiter": ",",            # Define comma as separator
    "field_optionally_enclosed_by": '"',  # Allow quotes around fields
    "infer_schema": True,              # Automatically detect data types
    "parse_header": True               # Use first row as column names
}).csv("@snowflake_recommender_system_with_implicit_bpr_stage")    # Read from Snowflake stage

# Display a formatted preview of the dataframe
print("\n=== ONLINE RETAIL DATASET PREVIEW ===")
spdf.show(10)                          # Show 10 rows of the dataframe
print("=============================\n")

## 🔢 Data Type Conversion for ML Compatibility

This code block performs targeted data type conversions to ensure our dataset works properly with machine learning algorithms. It systematically identifies decimal columns and converts them to double-precision floating-point format, addressing a common compatibility issue when working with Snowflake data in ML pipelines.

In [ ]:
# Convert decimal columns to double for ML compatibility
from snowflake.snowpark.types import DecimalType, DoubleType  # Import required Snowflake data types

# Find and convert all decimal columns in one pass
for col_name in [f.name for f in spdf.schema.fields if isinstance(f.datatype, DecimalType)]:  # Identify all decimal columns dynamically
    spdf = spdf.with_column(col_name, spdf[col_name].cast(DoubleType()))  # Cast each decimal column to double type

## 🏷️ Column Standardization and DataFrame Conversion

This code block handles the critical task of normalizing column names and converting our data to a pandas DataFrame. It removes special characters and quotes from column names while ensuring consistent naming conventions, which simplifies subsequent code development and improves readability.

In [ ]:
# Convert Snowpark DataFrame to pandas with clean column names
columns_mapping = {
    '"Customer ID"': "CustomerID",     # Remove quotes and space from customer identifier
    '"StockCode"': "StockCode",        # Standardize product code column name
    '"Description"': "Description",    # Clean product description column name
    '"Price"': "Price",                # Normalize price column name
    '"Quantity"': "Quantity",          # Standardize quantity column name
    '"Invoice"': "Invoice",            # Clean invoice identifier column name
    '"InvoiceDate"': "InvoiceDate",    # Normalize transaction date column name
    '"Country"': "Country"             # Standardize country column name
}

# Extract selected columns and rename in one operation
online_retail_data = spdf[list(columns_mapping.keys())].to_pandas()  # Select columns and convert to pandas
online_retail_data.columns = list(columns_mapping.values())          # Apply clean column names

online_retail_data

## 🛒 Customer Purchase History Display Function

This markdown describes a utility function that retrieves and displays purchase history for a specific customer. The function includes robust error handling for different CustomerID data types and provides attractive formatting to highlight key transaction details.

In [ ]:
def display_original_purchase_history(df, customer_id, limit=5):
    """
    Show top purchases for a specific customer with attractive formatting
    
    Parameters:
    -----------
    df : DataFrame
        The retail dataset containing purchase history
    customer_id : int or str
        The customer ID to search for
    limit : int, default=5
        Maximum number of purchases to display
        
    Returns:
    --------
    DataFrame or styled DataFrame
        The formatted purchase history
    """
    # Handle different CustomerID types
    try:
        # Convert CustomerID for comparison based on DataFrame's type
        df_type = type(df["CustomerID"].iloc[0])
        if df_type != type(customer_id):
            if isinstance(df["CustomerID"].iloc[0], (int, np.integer)):
                customer_id = int(customer_id)  # Convert to integer if DataFrame uses integers
            elif isinstance(df["CustomerID"].iloc[0], (float, np.float64)):  # Updated to np.float64
                customer_id = float(customer_id)  # Convert to float if DataFrame uses floats
            elif isinstance(df["CustomerID"].iloc[0], str):
                customer_id = str(customer_id)  # Convert to string if DataFrame uses strings
    except (IndexError, TypeError):
        # If type checking fails, continue with original customer_id
        pass
    
    # Try both direct and string-based comparison
    purchases = df[df["CustomerID"] == customer_id]  # First attempt direct comparison
    
    if purchases.empty:
        # Try string comparison if direct comparison failed
        purchases = df[df["CustomerID"].astype(str) == str(customer_id)]  # Fallback to string comparison
    
    # Select relevant columns if we found matches
    if not purchases.empty:
        purchases = purchases[
            ["CustomerID", "StockCode", "Description", "Quantity", "Invoice", "InvoiceDate"]
        ]  # Keep only the most relevant transaction details
    
    # Return empty dataframe with message if customer has no purchases
    if purchases.empty:
        print(f"⚠️ No purchase history found for Customer ID: {customer_id}")
        return purchases
    
    # Display styled header
    print(f"\n🛒 Purchase History for Customer ID: {customer_id}")
    print(f"Showing top {limit} of {len(purchases)} items")
    print("---------------------------------------------")
    
    # Format and display the dataframe with styling
    try:
        styled_df = purchases.head(limit).style.set_properties(**{
            'background-color': '#f5f5f5',  # Light gray background for readability
            'border-color': '#888888',      # Medium gray borders for definition
            'font-size': '11pt'             # Consistent font size
        }).format({
            'Quantity': '{:,.0f}',          # Format quantities as integers
            'InvoiceDate': '{:%Y-%m-%d}'    # Format dates consistently
        })
        
        # Handle potential hide_index compatibility issue with version check
        import pandas as pd
        if pd.__version__ >= '1.4.0':
            styled_df = styled_df.hide(axis="index")  # New method in pandas 1.4+
        else:
            try:
                styled_df = styled_df.hide_index()  # Older pandas versions
            except:
                pass  # Very old pandas version, skip hiding index
            
        return styled_df
    except Exception as e:
        print(f"Note: Basic formatting applied due to styling error: {e}")
        # Fallback to plain DataFrame if styling fails
        return purchases.head(limit)  # Return basic dataframe if styling fails

## 📊 Alternative Data Loading Method (Direct Excel Import)

This cell demonstrates an alternative approach to load retail data directly from Excel files without using Snowflake stages. This method is useful when working locally or when you need to process the Excel file before uploading to Snowflake.

In [ ]:
'''# This would take some time as we would be filling close 1 millon transactions from the downloaded file and creating a dataframe
raw_excel_data = pd.read_excel("online_retail_II.xlsx",
    sheet_name=None,                 # Load all sheets in the Excel file
    engine='openpyxl',               # Use the openpyxl engine for Excel processing 
    names=[                          # Define column names explicitly for consistency
        "Invoice",
        "StockCode",
        "Description",
        "Quantity",
        "InvoiceDate",
        "Price",
        "CustomerID",
        "Country",
    ],
)

# Since the file contains different two sheets from two years, let us combine and create a single dataframe
online_retail_data = pd.concat(raw_excel_data, axis=0, ignore_index=True)  # Merge all sheets into one dataset

# Print the recently created Panda Dataset's dimensionality and first five records
online_retail_data = online_retail_data[
    [                               # Reorder columns for better readability and consistency
        "CustomerID",
        "StockCode",
        "Description",
        "Price",
        "Quantity",
        "Invoice",
        "InvoiceDate",
        "Country",
    ]
]
'''

## 📝 Product Lookup Table Creation

This cell builds a reference lookup table that maps product codes to descriptions. This essential component allows our recommendation system to translate cryptic product codes into human-readable descriptions when displaying results to users.

In [ ]:
# Create product lookup table for recommendation display
stock_code_desc_look_up = (online_retail_data[["StockCode", "Description"]]  # Extract only product code and description columns
                          .drop_duplicates()                                 # Remove duplicate products
                          .astype({"StockCode": str}))                       # Ensure consistent string format for codes

print(f"[INFO]: Stock Code lookup table created with {len(stock_code_desc_look_up):,} unique products")

# Display sample with basic styling (compatible with older pandas versions)
preview_df = stock_code_desc_look_up.head(3)                                 # Get first 3 products for preview
try:
    # For newer pandas versions
    display(preview_df.style.set_properties(**{'background-color': '#f5f5f5'}).hide())  # Apply subtle background styling
except AttributeError:
    # Fallback for older pandas versions
    display(preview_df.style.set_properties(**{'background-color': '#f5f5f5'}))         # Simplified styling for compatibility

## 🎨 Data Visualization Utility Function

This cell defines a custom styling function that transforms plain dataframes into visually appealing tables. The function applies professional formatting to dates, currencies, and numbers while ensuring compatibility across different pandas versions.

In [ ]:
# Display a polished preview of the retail data
def styled_preview(df, rows=5):
    """Create an elegantly styled preview of the dataframe"""
    # Make a copy to avoid modifying the original
    preview_df = df.head(rows).copy()
    
    # Format currency without using \\$ in the formatter
    if "Price" in preview_df.columns:
        preview_df["Price"] = preview_df["Price"].apply(lambda x: f"£{x:.2f}" if pd.notnull(x) else "")  # Format prices as British pounds
    
    # Format specifications for other column types
    format_dict = {
        "CustomerID": "{:,.0f}",         # Whole numbers without decimals
        "Quantity": "{:,.0f}",           # Whole numbers without decimals
        "InvoiceDate": "{:%Y-%m-%d}"     # Formatted dates
    }
    
    # Apply styling with modern design
    try:
        styled = preview_df.style\
            .format(format_dict)\
            .set_properties(**{
                'font-family': 'Segoe UI, Helvetica, Arial, sans-serif',  # Professional font family
                'text-align': 'left',                                      # Left-aligned for better readability
                'border': '1px solid #e0e0e0',                             # Subtle borders
                'padding': '8px',                                          # Comfortable padding
                'background-color': '#ffffff'                              # Clean white background
            })\
            .set_table_styles([
                {'selector': 'thead th', 'props': [
                    ('background-color', '#f2f2f2'),                       # Light gray header background
                    ('color', '#333333'),                                  # Dark text for contrast
                    ('font-weight', 'bold'),                               # Bold headers
                    ('border-bottom', '2px solid #cccccc')                 # Distinct header border
                ]},
                {'selector': 'tbody tr:nth-child(even)', 'props': [
                    ('background-color', '#f9f9f9')                        # Zebra striping for rows
                ]}
            ])
        
        # Try to hide index - handle different pandas versions
        try:
            return styled.hide(axis='index')                               # Hide index in newer pandas versions
        except AttributeError:
            return styled                                                  # Fallback for older versions
    except Exception as e:
        # Fallback to simpler styling if advanced styling fails
        print(f"Using simplified styling due to: {str(e)}")
        return preview_df                                                  # Return plain preview if styling fails

# Display the styled preview
display(styled_preview(online_retail_data))                                # Show formatted retail data

## 📊 Dataset Profile and Quality Assessment

This cell performs a comprehensive examination of our retail dataset structure, displaying key metrics like record count, data types, memory usage, and missing values. These insights help us understand data quality issues before proceeding with analysis


In [ ]:
# Display dataset structure and quality metrics
print("\n" + "="*50)
print("📋 RETAIL DATASET PROFILE".center(50))
print("="*50)

# Show dataset dimensions with formatting
record_count = f"{len(online_retail_data):,}"
feature_count = f"{online_retail_data.shape[1]}"
print(f"📝 Records: {record_count} | 🔢 Features: {feature_count}")

# Create formatted table for data types and memory usage
print("\n" + "📊 DATATYPES AND MEMORY USAGE".center(50))
print("-"*50)
print(f"{'Column':<15} {'Type':<12} {'Memory':<10} {'% of Total':<12}")
print("-"*50)

total_memory = 0
for col in online_retail_data.columns:
    mem_usage = online_retail_data[col].memory_usage(deep=True) / (1024 * 1024)
    total_memory += mem_usage

for col, dtype in zip(online_retail_data.columns, online_retail_data.dtypes):
    mem_usage = online_retail_data[col].memory_usage(deep=True) / (1024 * 1024)
    pct_total = (mem_usage / total_memory) * 100
    print(f"{col:<15} {str(dtype):<12} {mem_usage:6.2f} MB  {pct_total:6.1f}%")

# Show missing values with visual indicators
print("\n" + "⚠️ MISSING VALUES SUMMARY".center(50))
print("-"*50)
missing_found = False

for col in online_retail_data.columns:
    missing = online_retail_data[col].isnull().sum()
    if missing > 0:
        missing_found = True
        pct = (missing / len(online_retail_data)) * 100
        bar_length = int(pct / 2)  # Scale for visual bar
        bar = '█' * bar_length + '░' * (50 - bar_length)
        print(f"{col:<12} {missing:>8,} missing ({pct:5.1f}%) {bar}")

if not missing_found:
    print("✅ No missing values detected in the dataset")
    
# Display quick statistics with improved formatting
print("\n" + "📈 KEY STATISTICS SUMMARY".center(50))
print("-"*50)
numeric_cols = online_retail_data.select_dtypes(include=['number']).columns

if len(numeric_cols) > 0:
    stats = online_retail_data[numeric_cols].describe()
    
    # Format each statistic with appropriate precision
    for col in stats.columns:
        col_max = stats[col].max()
        if abs(col_max) < 10:
            # For small numbers, show more decimals
            stats[col] = stats[col].apply(lambda x: f"{x:.3f}")
        elif abs(col_max) < 1000:
            # For medium numbers
            stats[col] = stats[col].apply(lambda x: f"{x:.2f}")
        else:
            # For large numbers, use comma separators
            stats[col] = stats[col].apply(lambda x: f"{int(x):,}" if x == int(x) else f"{x:,.1f}")
    
    print(stats.T.to_string())
else:
    print("No numeric columns available for statistics")

print("\n" + "="*50)

# Add key insights based on the data
print("\n🔍 KEY INSIGHTS:")
print(f"• Dataset spans {online_retail_data['InvoiceDate'].min().date()} to {online_retail_data['InvoiceDate'].max().date()}")
print(f"• Contains transactions from {online_retail_data['Country'].nunique()} different countries")
print(f"• {online_retail_data['CustomerID'].nunique():,} unique customers with purchasing history")
print(f"• Average purchase quantity: {online_retail_data['Quantity'].mean():.1f} items")

## 🧹 Data Cleaning Pipeline

A modular pipeline for preprocessing online retail data, built on scikit-learn. This cell implements a robust data cleaning pipeline that:

    ✅ Removes missing CustomerID records
    ✅ Standardizes data types for consistency
    ✅ Provides beautiful visual reporting
    ✅ Saves for production reuse

The pipeline follows ML best practices with proper train/test splitting and can deploy to both local and Snowflake environments.

The implementation produces detailed visual reports showing the cleaning impact and works seamlessly with recommendation system workflows.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline  # Back to sklearn Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
import joblib
import sys
from time import sleep

# Define the DataCleaner class at the module level
class DataCleaner(BaseEstimator, TransformerMixin):
    """
    Custom transformer for cleaning online retail data:
    - Removes records without CustomerID
    - Converts CustomerID to integer type
    - Provides detailed reporting on cleaning operations
    """
    
    def __init__(self, customer_id_col='CustomerID', report=True):
        self.customer_id_col = customer_id_col
        self.report = report
        self.initial_count = None
        self.initial_customers = None
        self.removed_count = None
        self.final_customers = None
        self.removal_percentage = None
        
    def fit(self, X, y=None):
        # Store initial metrics for reporting
        self.initial_count = len(X)
        self.initial_customers = X[self.customer_id_col].dropna().nunique()
        return self
        
    def transform(self, X, y=None):
        # Make a copy to avoid modifying the original dataframe
        df = X.copy()
        
        # Remove rows with missing CustomerID
        df = df.dropna(subset=[self.customer_id_col])
        
        # Convert CustomerID to integer type
        df[self.customer_id_col] = df[self.customer_id_col].astype(int)
        
        # Calculate result metrics
        self.removed_count = self.initial_count - len(df)
        self.removal_percentage = (self.removed_count / self.initial_count) * 100
        self.final_customers = df[self.customer_id_col].nunique()
        
        # Generate report if enabled
        if self.report:
            self._print_report()
            
        return df
    
    def _print_report(self):
        """Prints a detailed report of the cleaning operations and their impact"""
        print("\n" + "═"*70)
        print("🧹  DATA CLEANING PROCESS  🧹".center(70))
        print("═"*70)
        
        # Report initial status
        print(f"• Initial dataset: {self.initial_count:,} transactions")
        print(f"• Found {self.removed_count:,} transactions without CustomerID")
        
        # Report detailed results with visual indicators
        print("\n📊 CLEANING RESULTS:")
        print(f"  ✓ Removed {self.removed_count:,} incomplete records ({self.removal_percentage:.1f}% of data)")
        print(f"  ✓ Converted CustomerID to integer type for consistency")
        print(f"  ✓ Final dataset contains {self.initial_count - self.removed_count:,} valid transactions")
        print(f"  ✓ Dataset now has {self.final_customers:,} unique customers")
        
        # Add a visual representation of the data cleaning effect
        print("\n📈 DATA CLEANING IMPACT:")
        retention_rate = (self.initial_count - self.removed_count) / self.initial_count
        removal_rate = self.removed_count / self.initial_count
        print("  " + "█" * int(retention_rate * 40) + 
              "░" * int(removal_rate * 40) + f" {retention_rate:.1%} retained")
        print("═"*70 + "\n")


def print_section_header(title, width=70):
    """Print a beautifully formatted section header"""
    print("\n" + "═"*width)
    print(f"  {title}  ".center(width, "═"))
    print("═"*width)


def print_step_progress(step_name, pause=0.5):
    """Print a step progress indicator with a brief pause for visual effect"""
    print(f"\n⏳ {step_name}...", end="", flush=True)
    sleep(pause)
    print(" ✅")


# Make the class picklable by setting it as an attribute of the current module
setattr(sys.modules[__name__], 'DataCleaner', DataCleaner)


# Main execution flow
if __name__ == "__main__" or True:  # Always execute in notebook environment
    # Start with a clear header
    print_section_header("🔄 RECOMMENDER SYSTEM DATA PIPELINE 🔄")
    
    # Build the preprocessing pipeline
    print_step_progress("Building data processing pipeline")
    data_processing_pipeline = Pipeline(
        steps=[
            (
                "data_cleaner",
                DataCleaner(customer_id_col='CustomerID', report=False)  # Turn off immediate reporting
            )
        ]
    )
    
    # Define the pipeline file location
    PIPELINE_FILE = '/tmp/snowflake_implicit_bpr_recommender_processing_pipeline.joblib'
    
    # Split data for training/testing using scikit-learn
    print_step_progress("Splitting data into training and testing sets")
    training, testing = train_test_split(
        online_retail_data, test_size=0.2, random_state=111
    )
    
    # Process training data
    print_step_progress("Processing training data")
    training_processed = data_processing_pipeline.fit(training).transform(training)
    
    # Process testing data (correctly, without refitting)
    print_step_progress("Processing testing data") 
    testing_processed = data_processing_pipeline.transform(testing)
    
    # Process full dataset with reporting enabled (for visualization)
    print("\n📋 Processing full dataset with detailed reporting:")
    data_processing_pipeline.named_steps['data_cleaner'].report = True
    cleaned_data = data_processing_pipeline.transform(online_retail_data)
    online_retail_data = cleaned_data;
    
    # Save the pipeline
    print_step_progress("Saving pipeline to local file")
    joblib.dump(data_processing_pipeline, PIPELINE_FILE)
    
    # Create stage if it doesn't exist
    print_step_progress("Creating Snowflake stage")
    stage_name = "snowflake_implicit_bpr_recommender_processing_pipeline_stage"
    
    try:
        stage_result = session.sql(f"CREATE OR REPLACE STAGE {stage_name}").collect()
        
        # Upload to Snowflake stage
        print_step_progress("Uploading pipeline to Snowflake stage")
        upload_result = session.file.put(
            PIPELINE_FILE, 
            f"@{stage_name}", 
            overwrite=True
        )
        
        # Print upload details
        print_section_header("📤 FILE UPLOAD DETAILS 📤")
        
        if upload_result:
            # Display upload results in a formatted way
            upload_info = upload_result[0]
            print(f"• File name: {upload_info[0]}")
            print(f"• Compressed name: {upload_info[1]}")
            print(f"• Source size: {upload_info[2]:,} bytes")
            print(f"• Target size: {upload_info[3]:,} bytes")
            print(f"• Source compression: {upload_info[4]}")
            print(f"• Target compression: {upload_info[5]}")
            print(f"• Status: {upload_info[6]}")
            
            # Calculate compression ratio if applicable
            if upload_info[2] > 0:
                compression_ratio = (1 - (upload_info[3] / upload_info[2])) * 100
                print(f"• Compression ratio: {compression_ratio:.1f}%")
        else:
            print("⚠️ No upload details available")
            
    except NameError:
        print("\n⚠️ Snowflake session not available - skipping stage creation and file upload")
    
    # Final summary
    print_section_header("📊 PIPELINE RESULTS SUMMARY 📊")
    print(f"• Training data shape: {training_processed.shape}")
    print(f"• Testing data shape: {testing_processed.shape}")
    print(f"• Memory usage: {cleaned_data.memory_usage().sum() / 1024**2:.2f} MB")
    
    if 'session' in globals() or 'session' in locals():
        print(f"• Pipeline saved to: @{stage_name}")
    else:
        print(f"• Pipeline saved locally to: {PIPELINE_FILE}")
    
    # Display data types in a cleaner format
    print("\n📋 DATA TYPES AFTER PROCESSING:")
    for col, dtype in zip(cleaned_data.columns, cleaned_data.dtypes):
        print(f"  • {col}: {dtype}")
    
    print("\n" + "═"*70)
    print("🎉 PIPELINE EXECUTION COMPLETED SUCCESSFULLY 🎉".center(70))
    print("═"*70 + "\n")


In [ ]:
LS @snowflake_implicit_bpr_recommender_processing_pipeline_stage

In [ ]:
'''# Alternate to above pipeline code : Clean data by removing records without CustomerID and converting to correct type
print("\n" + "="*50)
print("🧹 DATA CLEANING PROCESS".center(50))
print("="*50)

# Track initial state for reporting
initial_count = len(online_retail_data)
initial_customers = online_retail_data['CustomerID'].dropna().nunique()

# Report initial status
print(f"• Initial dataset: {initial_count:,} transactions")
print(f"• Found {online_retail_data['CustomerID'].isna().sum():,} transactions without CustomerID")

# Remove rows with missing CustomerID
online_retail_data = online_retail_data.dropna(subset=['CustomerID'])  # Filter out records without customer ID
removed_count = initial_count - len(online_retail_data)                # Calculate how many records were removed

# Convert CustomerID to integer type
online_retail_data["CustomerID"] = online_retail_data.CustomerID.astype(int)  # Ensure consistent ID format

# Calculate result metrics
removal_percentage = (removed_count/initial_count) * 100
final_customers = online_retail_data['CustomerID'].nunique()

# Report detailed results with visual indicators
print("\n📊 CLEANING RESULTS:")
print(f"✓ Removed {removed_count:,} incomplete records ({removal_percentage:.1f}% of data)")
print(f"✓ Converted CustomerID to integer type for consistency")
print(f"✓ Final dataset contains {len(online_retail_data):,} valid transactions")
print(f"✓ Dataset now has {final_customers:,} unique customers")

# Show data type summary after cleaning
print("\n📋 CLEANED DATA TYPES:")
for col, dtype in zip(online_retail_data.columns, online_retail_data.dtypes):
    print(f"  • {col}: {dtype}")

# Add a visual representation of the data cleaning effect
print("\n📈 DATA CLEANING IMPACT:")
print("Transactions:  " + "█" * int(len(online_retail_data)/initial_count * 40) + 
      "░" * int(removed_count/initial_count * 40) + f" {len(online_retail_data)/initial_count:.1%} retained")

print("="*50)

'''

## 🔍 Dataset Dimension Analysis

This cell analyzes the core dimensions of our retail dataset, providing a quantitative overview of transactions, products, customers, and geographic distribution. Understanding these metrics helps calibrate our recommendation approach.

In [ ]:
# Analyze dataset dimensions and unique entities
print("\n" + "="*60)
print("🔍 RETAIL DATASET DIMENSION ANALYSIS".center(60))
print("="*60)

# Calculate key metrics
metrics = {
    "Transactions": online_retail_data['Invoice'].nunique(),              # Count unique invoice numbers
    "Unique Products": online_retail_data['StockCode'].nunique(),         # Count distinct product codes
    "Unique Customers": online_retail_data['CustomerID'].nunique(),       # Count individual customers
    "Missing Customer IDs": online_retail_data['CustomerID'].isnull().sum(),  # Count records with no customer ID
    "Countries": online_retail_data['Country'].nunique(),                 # Count distinct countries
    "Date Range": f"{online_retail_data['InvoiceDate'].min().date()} to {online_retail_data['InvoiceDate'].max().date()}"
}

# Display with enhanced formatting
print("\n📊 CORE METRICS:")
print("-"*60)
print(f"{'Category':<22} {'Value':<15} {'Notes'}")
print("-"*60)

# Format and display each metric with additional context
print(f"{'Transactions':<22} {metrics['Transactions']:,} {'(Unique invoice numbers)'}")
print(f"{'Unique Products':<22} {metrics['Unique Products']:,} {'(Distinct product codes)'}")
print(f"{'Unique Customers':<22} {metrics['Unique Customers']:,} {'(Individual buyers)'}")
print(f"{'Countries':<22} {metrics['Countries']} {'(Geographic distribution)'}")
print(f"{'Date Range':<22} {metrics['Date Range']} {'(Transaction period)'}")

# Calculate average transactions per customer
avg_trans = online_retail_data.groupby('CustomerID')['Invoice'].nunique().mean()
print(f"{'Avg Trans/Customer':<22} {avg_trans:.1f} {'(Customer engagement metric)'}")

# Add top countries by transaction volume with visual indicators
top_countries = online_retail_data['Country'].value_counts().head(5)
total_transactions = len(online_retail_data)

print("\n🌍 TOP 5 COUNTRIES BY TRANSACTION VOLUME:")
print("-"*60)
print(f"{'Country':<15} {'Transactions':>12} {'Percentage':>12} {'Distribution'}")
print("-"*60)

for country, count in top_countries.items():
    percentage = count / total_transactions * 100
    bar_length = int(percentage / 2)  # Scale for visual bar
    bar = '█' * bar_length
    print(f"{country:<15} {count:>12,} {percentage:>11.1f}% {bar}")

# Add product category insights (using first words of descriptions as proxy for categories)
print("\n📦 PRODUCT CATEGORY INSIGHTS:")
# Extract first word as rough category and count frequencies
online_retail_data['Category'] = online_retail_data['Description'].str.split().str[0]
top_categories = online_retail_data['Category'].value_counts().head(5)

print("-"*60)
for category, count in top_categories.items():
    print(f"• {category:<15} {count:,} items")

print("-"*60)

## 📊 Country Performance Analysis Dashboard

This cell creates a comprehensive visual dashboard that analyzes retail performance metrics across different countries. The dashboard combines revenue analysis, customer value assessment, time trends, and product category distribution into a single informative visualization.

In [ ]:
"""
Retail Data Insights - Streamlined country performance analysis
"""

def quick_retail_insights(df, top_n=6):
    """Generate comprehensive retail insights with minimal code"""
    # Ensure required columns exist and calculate order values
    if 'total_cost' not in df.columns:
        df['total_cost'] = df['Quantity'] * df['Price']  # Calculate revenue per transaction
    
    # Print key metrics
    print(f"📊 RETAIL INSIGHTS | {len(df):,} transactions | {df['CustomerID'].nunique():,} customers | {df['Country'].nunique()} countries")
    print(f"💰 Total Revenue: \\${df['total_cost'].sum():,.2f} | Avg Order: \\${df.groupby('Invoice')['total_cost'].sum().mean():,.2f}")
    
    # Find cancelled transactions
    cancelled = df[df['Invoice'].astype(str).str.startswith('C')]  # Identify cancelled orders (typically prefixed with 'C')
    print(f"❌ Cancelled Orders: {len(cancelled):,} ({len(cancelled)/len(df):.1%})")
    
    # Country metrics - calculate key performance indicators by country
    metrics = df.groupby("Country").agg(
        revenue=('total_cost', 'sum'),                      # Total revenue by country
        avg_order=('total_cost', lambda x: x.mean()),       # Average order value
        orders=('Invoice', 'nunique'),                      # Number of orders
        customers=('CustomerID', 'nunique')                 # Unique customers
    ).sort_values(by="revenue", ascending=False)            # Sort by highest revenue
    
    metrics['share'] = metrics['revenue'] / metrics['revenue'].sum() * 100           # Calculate market share percentage
    metrics['avg_customer_value'] = metrics['revenue'] / metrics['customers']        # Calculate customer lifetime value
    top_countries = metrics.head(top_n)                                              # Select top performing countries
    
    # Create figure with optimized layout
    fig = plt.figure(figsize=(16, 12))                      # Create large figure for dashboard
    plt.style.use('seaborn-v0_8-whitegrid')                 # Set consistent styling
    
    # === COMBINED VISUALIZATION ===
    # 1. Main revenue chart with integrated metrics
    ax1 = plt.subplot2grid((2, 3), (0, 0), colspan=2)       # Create subplot in grid layout
    
    # Plot bars with custom styling
    palette = sns.color_palette("viridis", top_n)           # Generate color palette
    bars = ax1.barh(top_countries.index, top_countries['revenue'], color=palette)  # Create horizontal bar chart
    
    # Add rich data labels
    for i, bar in enumerate(bars):
        country = top_countries.index[i]
        revenue = top_countries['revenue'].iloc[i]
        share = top_countries['share'].iloc[i]
        customers = top_countries['customers'].iloc[i]
        
        # Add revenue and market share
        ax1.text(
            revenue + revenue*0.01, i,                      # Position text at end of bar
            f"\\${revenue:,.0f} ({share:.1f}%)",            # Format revenue with \$ and add percentage
            va='center', fontweight='bold'                  # Center align vertically and make bold
        )
        
        # Add customer count inside bar
        if revenue > top_countries['revenue'].max() * 0.25:  # Only add text if bar is long enough
            ax1.text(
                revenue * 0.5, i,                            # Position text in middle of bar
                f"{customers:,} customers",                  # Show customer count with comma formatting
                va='center', ha='center',                    # Center align text
                color='white', fontweight='bold'             # White text for visibility on colored bar
            )
    
    ax1.set_title("Country Revenue Performance", fontsize=14, fontweight='bold')
    ax1.set_xlabel("Revenue (\\$)", fontsize=12)
    
    # 2. Customer value vs order size scatter
    ax2 = plt.subplot2grid((2, 3), (0, 2))                  # Create subplot for scatter plot
    scatter = ax2.scatter(
        top_countries['avg_order'],                         # X-axis: average order value
        top_countries['avg_customer_value'],                # Y-axis: customer lifetime value 
        s=top_countries['orders']/10,                       # Size points by order count
        c=range(len(top_countries)),                        # Color by revenue rank
        cmap='viridis', alpha=0.8                           # Match color scheme with bar chart
    )
    
    # Add country labels
    for country in top_countries.index:
        ax2.annotate(
            country,                                         # Label points with country name
            (top_countries.loc[country, 'avg_order'],        # X position
             top_countries.loc[country, 'avg_customer_value']), # Y position
            fontsize=9, fontweight='bold'                    # Make text readable
        )
    
    ax2.set_title("Customer Value Analysis", fontsize=14, fontweight='bold')
    ax2.set_xlabel("Avg Order Value (\\$)")
    ax2.set_ylabel("Avg Customer Value (\\$)")
    ax2.grid(True, linestyle='--', alpha=0.7)               # Add subtle grid lines
    
    # 3. Time series for top 3 markets
    ax3 = plt.subplot2grid((2, 3), (1, 0), colspan=2)       # Wide subplot for time series
    
    # Create monthly time series
    df['month'] = pd.to_datetime(df['InvoiceDate']).dt.to_period('M').dt.to_timestamp()  # Convert dates to months
    
    # Plot top 3 countries
    for i, country in enumerate(top_countries.index[:3]):    # Loop through top 3 countries
        country_data = df[df['Country'] == country].groupby('month')['total_cost'].sum()  # Get monthly revenue
        ax3.plot(
            country_data.index, country_data.values,         # Plot time series
            marker='o', linewidth=2, label=country, color=palette[i]  # Style with markers and matching colors
        )
        
        # Add last value annotation
        if not country_data.empty:
            last_month = country_data.index[-1]              # Get most recent month
            last_value = country_data.values[-1]             # Get final revenue value
            ax3.annotate(
                f"\\${last_value:,.0f}",                      # Format as currency
                (last_month, last_value),                    # Position at end of line
                xytext=(10, 0), textcoords='offset points'   # Offset slightly for readability
            )
    
    ax3.set_title("Monthly Revenue Trends - Top Markets", fontsize=14, fontweight='bold')
    ax3.set_ylabel("Monthly Revenue (\\$)")
    ax3.legend()                                            # Add country legend
    ax3.grid(True, alpha=0.3)                               # Add subtle grid
    
    # 4. Category mix by country (new insight)
    ax4 = plt.subplot2grid((2, 3), (1, 2))                  # Create subplot for category analysis
    
    # Use product description first word as category proxy
    df['category'] = df['Description'].astype(str).str.split().str[0]  # Extract first word as category
    top_cats = df['category'].value_counts().head(5).index   # Get top 5 categories
    
    # Get category distribution for top 3 countries
    cat_data = []
    for country in top_countries.index[:3]:                  # For each top country
        country_cats = df[df['Country'] == country]['category'].value_counts()  # Count categories
        cat_data.append([country_cats.get(cat, 0) for cat in top_cats])  # Create list of category counts
    
    # Create normalized stacked bars
    cat_data_norm = np.array(cat_data) / np.array(cat_data).sum(axis=1)[:, np.newaxis]  # Normalize to percentages
    ax4.bar(top_countries.index[:3], np.ones(3), color='lightgray')  # Create background bars
    bottom = np.zeros(3)                                     # Initialize bottom position
    
    for i, cat in enumerate(top_cats):                       # For each category
        ax4.bar(
            top_countries.index[:3], cat_data_norm[:, i],    # Plot proportional segment
            bottom=bottom, label=cat, alpha=0.8              # Stack on previous segments
        )
        bottom += cat_data_norm[:, i]                        # Update bottom for next category
    
    ax4.set_title("Product Category Mix", fontsize=14, fontweight='bold')
    ax4.set_ylim(0, 1)                                       # Set y-axis from 0 to 100%
    ax4.set_ylabel("Category Share")
    ax4.legend(fontsize=8)                                   # Add small legend
    
    # Add overall title and insights
    plt.suptitle("Country Performance Dashboard", fontsize=18, fontweight='bold', y=0.98)
    
    # Add insights box with key takeaways
    insights = "\n".join([
        "📌 " + country + ": " + 
        f"\\${metrics.loc[country, 'revenue']:,.0f} revenue, " + 
        f"{metrics.loc[country, 'customers']:,} customers"
        for country in metrics.index[:3]                     # Create summary for top 3 countries
    ])
    
    plt.figtext(0.5, 0.01, insights, ha='center', bbox={     # Add text box at bottom
        'facecolor': 'lightyellow', 'alpha': 0.9, 'pad': 5,
        'boxstyle': 'round,pad=0.5'                          # Style as a note
    })
    
    plt.tight_layout(rect=[0, 0.05, 1, 0.95])                # Adjust layout for readability
    return fig

# Generate the dashboard
dashboard = quick_retail_insights(online_retail_data)        # Create visualization with our data
plt.show()   

## 🤖 Recommendation System Construction and Evaluation

This comprehensive cell implements a complete product recommendation engine using Bayesian Personalized Ranking (BPR). It includes data preparation, model training, evaluation metrics calculation, performance visualization, and a recommendation generator function for individual customers.

In [ ]:
"""
Retail Recommender System with Performance Metrics
"""

def build_recommendation_system(df, test_size=0.2, factors=50, iterations=50, 
                              learning_rate=0.05, regularization=0.01):
    """Build and evaluate a recommendation system with comprehensive metrics"""
    print("📊 RETAIL RECOMMENDER | Building recommendation system...")
    
    # Track performance metrics
    metrics = {}                                      # Dictionary to store all performance metrics
    start_time = time.time()                          # Record start time for overall process
    
    # Step 1: Prepare data
    print("1️⃣ Preparing data...")
    prep_start = time.time()                          # Track time for data preparation step
    
    # Focus on required columns with positive quantities
    df_prep = df[['CustomerID', 'StockCode', 'Quantity']].copy()  # Extract relevant columns
    df_prep = df_prep[(df_prep['CustomerID'].notna()) &           # Filter out rows with missing IDs
                     (df_prep['StockCode'].notna()) &             # Filter out rows with missing products
                     (df_prep['Quantity'] > 0)]                   # Only keep positive quantities (purchases)
    
    # Convert to strings for encoding
    df_prep['CustomerID'] = df_prep['CustomerID'].astype(str)     # Ensure consistent formatting for encoding
    df_prep['StockCode'] = df_prep['StockCode'].astype(str)       # Convert product codes to strings
    
    # Create encoders
    user_encoder = LabelEncoder()                     # Encoder for customer IDs
    item_encoder = LabelEncoder()                     # Encoder for product codes
    
    df_prep['user'] = user_encoder.fit_transform(df_prep['CustomerID'])  # Map customers to integer indices
    df_prep['item'] = item_encoder.fit_transform(df_prep['StockCode'])   # Map products to integer indices
    
    # Record data metrics
    metrics['total_users'] = len(user_encoder.classes_)            # Number of unique customers
    metrics['total_items'] = len(item_encoder.classes_)            # Number of unique products
    metrics['total_interactions'] = len(df_prep)                   # Total purchase records
    metrics['data_prep_time'] = time.time() - prep_start           # Time taken for data preparation
    
    print(f"   ✓ Found {metrics['total_users']:,} customers and {metrics['total_items']:,} products")
    print(f"   ✓ Using {metrics['total_interactions']:,} purchase interactions")
    
    # Step 2: Create train/test split for evaluation
    print("2️⃣ Splitting data for evaluation...")
    split_start = time.time()                         # Track time for train/test splitting
    
    # Group by user-item for unique interactions
    interactions = df_prep.groupby(['user', 'item'])['Quantity'].sum().reset_index()  # Aggregate purchases
    
    # Split at the user level for better evaluation
    users = df_prep['user'].unique()                  # Get all unique user indices
    train_users, test_users = train_test_split(users, test_size=test_size, random_state=42)  # Split users
    
    # Create train and test sets
    train_df = df_prep[df_prep['user'].isin(train_users)]  # Training data from training users
    test_df = df_prep[df_prep['user'].isin(test_users)]    # Test data from test users
    
    # Create matrices
    n_users = df_prep['user'].max() + 1               # Total number of users (0-indexed)
    n_items = df_prep['item'].max() + 1               # Total number of items (0-indexed)
    
    train_interactions = train_df.groupby(['user', 'item'])['Quantity'].sum().reset_index()  # Aggregate
    train_matrix = coo_matrix(
        (train_interactions['Quantity'].astype(float),                # Values (purchase quantities)
         (train_interactions['user'], train_interactions['item'])),   # Row and column indices
        shape=(n_users, n_items)                                      # Matrix dimensions
    ).tocsr()                                         # Convert to CSR format for efficient operations
    
    test_interactions = test_df.groupby(['user', 'item'])['Quantity'].sum().reset_index()  # Aggregate
    test_matrix = coo_matrix(
        (test_interactions['Quantity'].astype(float),                 # Values (purchase quantities) 
         (test_interactions['user'], test_interactions['item'])),     # Row and column indices
        shape=(n_users, n_items)                                      # Match train matrix dimensions
    ).tocsr()                                         # Convert to CSR format for efficient operations
    
    metrics['split_time'] = time.time() - split_start  # Time taken for data splitting
    print(f"   ✓ Created training set with {len(train_df):,} interactions")
    print(f"   ✓ Created test set with {len(test_df):,} interactions for evaluation")
    
    # Step 3: Train model
    print("3️⃣ Training recommendation model...")
    train_start = time.time()                         # Track time for model training
    
    model = BayesianPersonalizedRanking(
        factors=factors,                              # Number of latent factors (complexity)
        iterations=iterations,                        # Training iterations
        learning_rate=learning_rate,                  # Step size for gradient descent
        regularization=regularization,                # Regularization to prevent overfitting
        random_state=42                               # Ensure reproducible results
    )
    model.fit(train_matrix.T)                         # Train on transposed matrix (implicit expects item-user format)
    
    metrics['training_time'] = time.time() - train_start  # Time taken for model training
    print(f"   ✓ Model trained in {metrics['training_time']:.2f} seconds")
    print(f"   ✓ Using {factors} latent factors with {iterations} iterations")
    
    # Step 4: Evaluate model
    print("4️⃣ Evaluating model performance...")
    eval_start = time.time()                          # Track time for evaluation
    
    # Calculate precision@k and recall@k for test users
    k_values = [5, 10, 20]                            # Different numbers of recommendations to evaluate
    precision_at_k = {k: [] for k in k_values}        # Store precision for each k
    recall_at_k = {k: [] for k in k_values}           # Store recall for each k
    
    # Sample users for evaluation (up to 100 for speed)
    eval_users = np.random.choice(test_users, min(100, len(test_users)), replace=False)  # Sample users
    
    for user in eval_users:
        # Get items this user has interacted with in test set
        actual_items = set(test_df[test_df['user'] == user]['item'])  # Items the user actually purchased
        if not actual_items:
            continue                                  # Skip users with no test interactions
            
        # Get recommendations
        try:
            recs, scores = model.recommend(user, train_matrix[user], N=max(k_values))  # Generate recommendations
        except:
            continue                                  # Skip if recommendation fails
            
        # Calculate metrics
        for k in k_values:
            recommended_items = set(recs[:k])         # Top-k recommendations
            relevant_and_recommended = len(recommended_items.intersection(actual_items))  # Hits
            
            # Precision = relevant & recommended / recommended
            precision = relevant_and_recommended / k if k > 0 else 0  # Proportion of relevant recommendations
            precision_at_k[k].append(precision)
            
            # Recall = relevant & recommended / relevant
            recall = relevant_and_recommended / len(actual_items) if actual_items else 0  # Proportion of found items
            recall_at_k[k].append(recall)
    
    # Calculate average metrics
    for k in k_values:
        metrics[f'precision@{k}'] = np.mean(precision_at_k[k]) if precision_at_k[k] else 0  # Average precision
        metrics[f'recall@{k}'] = np.mean(recall_at_k[k]) if recall_at_k[k] else 0           # Average recall
    
    # Calculate coverage
    all_recommended = set()                           # Track all items that get recommended
    sample_users = np.random.choice(df_prep['user'].unique(), min(500, n_users), replace=False)  # Sample users
    
    for user in sample_users:
        try:
            recs, _ = model.recommend(user, train_matrix[user], N=10)  # Get recommendations for user
            all_recommended.update(recs)                               # Add to set of recommended items
        except:
            continue                                  # Skip errors
    
    metrics['catalog_coverage'] = len(all_recommended) / n_items  # Proportion of catalog that gets recommended
    metrics['eval_time'] = time.time() - eval_start               # Time taken for evaluation
    
    # Step 5: Save model
    joblib.dump(model, "snowflake_recommender_system_with_implicit_bpr_model.joblib", compress=3)            # Save trained model
    joblib.dump(user_encoder, "user_encoder.joblib", compress=3)              # Save user ID mapping
    joblib.dump(item_encoder, "item_encoder.joblib", compress=3)              # Save item ID mapping
    
    metrics['total_time'] = time.time() - start_time                          # Total processing time
    
    # Print summary
    print("\n✅ RECOMMENDATION SYSTEM BUILT SUCCESSFULLY")
    print(f"Total processing time: {metrics['total_time']:.2f} seconds")
    
    # Return all components
    return {
        'model': model,                               # Trained recommendation model
        'user_encoder': user_encoder,                 # Mapping from customer IDs to indices
        'item_encoder': item_encoder,                 # Mapping from product codes to indices
        'train_matrix': train_matrix,                 # Training data in matrix form
        'test_matrix': test_matrix,                   # Test data in matrix form
        'metrics': metrics                            # Performance metrics
    }

def visualize_model_performance(rec_system):
    """Create visualization of model performance metrics"""
    metrics = rec_system['metrics']                   # Extract metrics from recommendation system
    
    # Create figure for metrics
    plt.style.use('seaborn-v0_8-whitegrid')          # Clean visualization style
    fig, axes = plt.subplots(2, 2, figsize=(16, 12)) # 2x2 grid of visualizations
    
    # 1. Precision and Recall Plot
    ax1 = axes[0, 0]                                  # First subplot location
    k_values = [5, 10, 20]                            # Different recommendation list sizes
    precision_values = [metrics.get(f'precision@{k}', 0) for k in k_values]  # Get precision for each k
    recall_values = [metrics.get(f'recall@{k}', 0) for k in k_values]        # Get recall for each k
    
    width = 0.35                                      # Bar width
    x = np.arange(len(k_values))                      # Bar positions
    ax1.bar(x - width/2, precision_values, width, label='Precision', color='#5DA5DA')  # Precision bars
    ax1.bar(x + width/2, recall_values, width, label='Recall', color='#FAA43A')        # Recall bars
    
    ax1.set_xlabel('k (number of recommendations)')
    ax1.set_ylabel('Recommendations Score')
    ax1.set_title('Precision and Recall at k', fontweight='bold')
    ax1.set_xticks(x)
    ax1.set_xticklabels([f'k={k}' for k in k_values])
    ax1.legend()
    
    # Add value labels
    for i, v in enumerate(precision_values):
        ax1.text(i - width/2, v + 0.01, f'{v:.3f}', ha='center')  # Label precision values
    for i, v in enumerate(recall_values):
        ax1.text(i + width/2, v + 0.01, f'{v:.3f}', ha='center')  # Label recall values
    
    # 2. Coverage and User/Item Stats
    ax2 = axes[0, 1]                                  # Second subplot location
    coverage = metrics.get('catalog_coverage', 0)     # Get catalog coverage metric
    
    # Create gauge-style chart for coverage
    coverage_pct = coverage * 100                     # Convert to percentage
    gauge = ax2.pie([coverage_pct, 100-coverage_pct], 
                   startangle=90, colors=['#60BD68', '#EEEEEE'],    # Green for covered, gray for uncovered
                   wedgeprops=dict(width=0.3, edgecolor='w'))       # Donut chart style
    
    # Add labels and annotations
    ax2.text(0, 0, f"{coverage_pct:.1f}%", ha='center', va='center', fontsize=24, fontweight='bold')  # Center text
    ax2.text(0, -1.5, "Catalog Coverage", ha='center', fontsize=14) # Label below chart
    ax2.set_title('Model Coverage', fontweight='bold')
    
    # 3. Training Performance
    ax3 = axes[1, 0]                                  # Third subplot location
    
    # Time metrics
    time_metrics = [
        ('Data Prep', metrics.get('data_prep_time', 0)),   # Time for data preparation
        ('Split', metrics.get('split_time', 0)),           # Time for train/test split
        ('Training', metrics.get('training_time', 0)),     # Time for model training
        ('Evaluation', metrics.get('eval_time', 0))        # Time for model evaluation
    ]
    
    # Create horizontal bars for timing
    y_pos = np.arange(len(time_metrics))              # Bar positions
    times = [t[1] for t in time_metrics]              # Time values
    labels = [t[0] for t in time_metrics]             # Step labels
    
    bars = ax3.barh(y_pos, times, color=plt.cm.viridis(np.linspace(0, 0.8, len(times))))  # Colorful bars
    
    # Add time labels
    for i, v in enumerate(times):
        ax3.text(v + 0.1, i, f'{v:.2f}s', va='center')  # Label with seconds
    
    ax3.set_yticks(y_pos)
    ax3.set_yticklabels(labels)
    ax3.set_xlabel('Time (seconds)')
    ax3.set_title('Processing Time Breakdown', fontweight='bold')
    
    # 4. Data Statistics
    ax4 = axes[1, 1]                                  # Fourth subplot location
    
    # Prepare data stats
    data_stats = {
        'Customers': metrics.get('total_users', 0),         # Number of unique customers
        'Products': metrics.get('total_items', 0),          # Number of unique products
        'Interactions': metrics.get('total_interactions', 0) # Total purchase records
    }
    
    # Create horizontal bars with log scale for better visualization
    labels = list(data_stats.keys())                  # Category labels
    values = list(data_stats.values())                # Count values
    
    y_pos = np.arange(len(labels))                    # Bar positions
    bars = ax4.barh(y_pos, values, color=plt.cm.viridis(np.linspace(0.2, 1, len(values))))  # Colorful bars
    
    # Add value labels
    for i, v in enumerate(values):
        ax4.text(v + v*0.01, i, f'{v:,}', va='center')  # Label with formatted numbers
    
    ax4.set_yticks(y_pos)
    ax4.set_yticklabels(labels)
    ax4.set_xscale('log')                             # Log scale to handle different magnitudes
    ax4.set_title('Dataset Statistics', fontweight='bold')
    
    # Add overall title
    plt.suptitle('Recommendation System Performance Metrics', fontsize=20, fontweight='bold', y=0.98)
    
    # Add model specs as text box
    model_specs = "\n".join([
        f"• Algorithm: Bayesian Personalized Ranking",      # Model type
        f"• Latent Factors: {rec_system['model'].factors}", # Complexity parameter
        f"• Learning Rate: {rec_system['model'].learning_rate}", # Training parameter
        f"• Training Iterations: {rec_system['model'].iterations}", # Training thoroughness
        f"• Regularization: {rec_system['model'].regularization}"   # Overfitting control
    ])
    
    plt.figtext(0.5, 0.01, model_specs, ha='center', bbox={  # Add text box at bottom
        'facecolor': '#F5F5F5', 'alpha': 0.9, 'pad': 5,
        'boxstyle': 'round,pad=0.5'
    })
    
    plt.tight_layout(rect=[0, 0.05, 1, 0.95])         # Adjust spacing for readability
    return fig

def recommend_for_customer(customer_id, rec_system, num_recs=5):
    """Generate recommendations for a specific customer with visualization"""
    model = rec_system['model']                        # Trained recommendation model
    user_encoder = rec_system['user_encoder']          # Customer ID mapping
    item_encoder = rec_system['item_encoder']          # Product code mapping
    train_matrix = rec_system['train_matrix']          # User-item interaction matrix
    
    # Check if customer exists
    if customer_id not in user_encoder.classes_:
        print(f"⚠️ Customer {customer_id} not found. Showing popular items instead.")
        # Get popular items
        item_popularity = np.asarray(train_matrix.sum(axis=0)).flatten()  # Sum interactions by item
        popular_items = np.argsort(-item_popularity)[:num_recs]  # Sort by popularity (descending)
        
        # Create recommendation dataframe
        recs_df = pd.DataFrame({
            'StockCode': item_encoder.classes_[popular_items],  # Convert indices to product codes
            'Score': np.linspace(0.5, 0.3, num_recs),          # Generate declining scores
            'Type': 'Popular Item'                              # Indicate recommendation type
        })
        return recs_df
    
    # Get user index
    user_idx = np.where(user_encoder.classes_ == customer_id)[0][0]  # Look up customer index
    
    # Get user's purchased items
    user_items = train_matrix[user_idx]               # Extract user's row from interaction matrix
    
    try:
        # Get recommendations with safety checks for index bounds
        recommended_items, scores = model.recommend(
            user_idx, user_items, N=num_recs,         # Request specific number of recommendations
            filter_already_liked_items=True            # Don't recommend already purchased items
        )
        
        # Make sure indices are within bounds
        valid_mask = recommended_items < len(item_encoder.classes_)  # Check for valid product indices
        if not all(valid_mask):
            print(f"⚠️ Some recommendations were out of bounds. Filtering to valid items only.")
            recommended_items = recommended_items[valid_mask]  # Keep only valid indices
            scores = scores[valid_mask]                        # Keep corresponding scores
        
        # Create recommendation dataframe
        recs_df = pd.DataFrame({
            'StockCode': item_encoder.classes_[recommended_items],  # Convert indices to product codes
            'Score': scores,                                        # Include predicted scores
            'Type': 'Personalized'                                  # Indicate recommendation type
        })
        
    except Exception as e:
        print(f"⚠️ Error generating recommendations: {str(e)}")
        print("Showing popular items instead.")
        
        # Fall back to popular items
        item_popularity = np.asarray(train_matrix.sum(axis=0)).flatten()  # Calculate item popularity
        popular_indices = np.argsort(-item_popularity)                    # Sort by popularity
        
        # Filter to ensure indices are within bounds
        valid_indices = [i for i in popular_indices if i < len(item_encoder.classes_)][:num_recs]
        
        # Create recommendation dataframe with popular items
        recs_df = pd.DataFrame({
            'StockCode': item_encoder.classes_[valid_indices],  # Convert indices to product codes
            'Score': np.linspace(0.5, 0.3, len(valid_indices)), # Generate declining scores
            'Type': 'Popular Item (Fallback)'                   # Indicate fallback recommendation
        })
    
    return recs_df

# Run the full pipeline
print("Building and evaluating recommendation system...")
recommendation_system = build_recommendation_system(online_retail_data)  # Build complete recommendation system

# Visualize performance
print("\nGenerating performance visualizations...")
performance_vis = visualize_model_performance(recommendation_system)  # Create performance dashboard
plt.show()                                                            # Display visualization

# Generate sample recommendations
print("\nSample recommendations for a customer:")
sample_customer = recommendation_system['user_encoder'].classes_[0]  # Get first customer from dataset
recommendations = recommend_for_customer(sample_customer, recommendation_system, num_recs=5)  # Generate recommendations
display(recommendations)

## 🛍️ Customer Product Recommendation Function

This enhanced recommendation function generates personalized product suggestions for any customer, adding detailed product descriptions from the original dataset and handling various edge cases for improved reliability.

In [ ]:
def recommend_for_customer(customer_id, rec_system, df, num_recs=5):
    """Generate recommendations for a specific customer with visualization"""
    model = rec_system['model']                        # Access the trained recommendation model
    user_encoder = rec_system['user_encoder']          # Mapping between customer IDs and internal indices
    item_encoder = rec_system['item_encoder']          # Mapping between product codes and internal indices
    train_matrix = rec_system['train_matrix']          # User-item interaction matrix from training data
    
    # Check if customer exists in our training data
    if customer_id not in user_encoder.classes_:
        print(f"⚠️ Customer {customer_id} not found. Showing popular items instead.")
        # Get popular items as fallback recommendation strategy
        item_popularity = np.asarray(train_matrix.sum(axis=0)).flatten()  # Sum interactions by item
        popular_items = np.argsort(-item_popularity)[:num_recs]           # Get top-N most popular items
        
        # Create recommendation dataframe with popular items
        recs_df = pd.DataFrame({
            'StockCode': item_encoder.classes_[popular_items],            # Map indices back to product codes
            'Score': np.linspace(0.5, 0.3, num_recs),                     # Assign declining confidence scores
            'Type': 'Popular Item'                                        # Mark as popularity-based recommendation
        })
        
        # Add customer ID to all recommendations
        recs_df['CustomerID'] = customer_id                               # Include target customer ID
        
        # Merge with original dataframe to get descriptions
        # Get unique stock codes and descriptions
        stock_desc_df = df[['StockCode', 'Description']].drop_duplicates()  # Create lookup table
        recs_df = recs_df.merge(stock_desc_df, on='StockCode', how='left')  # Join to get descriptions
        
        # Reorganize columns
        recs_df = recs_df[['CustomerID', 'StockCode', 'Description', 'Type', 'Score']]
        
        return recs_df
    
    # Get user index for existing customers
    user_idx = np.where(user_encoder.classes_ == customer_id)[0][0]       # Convert customer ID to matrix index
    
    # Get user's purchased items
    user_items = train_matrix[user_idx]                                   # Extract user's interaction profile
    
    try:
        # Get recommendations with safety checks for index bounds
        recommended_items, scores = model.recommend(
            user_idx, user_items, N=num_recs,                             # Request specific number of recommendations
            filter_already_liked_items=True                               # Don't recommend items already purchased
        )
        
        # Make sure indices are within bounds
        valid_mask = recommended_items < len(item_encoder.classes_)        # Check for valid indices
        if not all(valid_mask):
            print(f"⚠️ Some recommendations were out of bounds. Filtering to valid items only.")
            recommended_items = recommended_items[valid_mask]              # Filter to valid indices
            scores = scores[valid_mask]                                    # Filter corresponding scores
        
        # Create recommendation dataframe
        recs_df = pd.DataFrame({
            'StockCode': item_encoder.classes_[recommended_items],         # Map indices to product codes
            'Score': scores,                                               # Include confidence scores
            'Type': 'Personalized'                                         # Mark as personalized recommendations
        })
        
    except Exception as e:
        print(f"⚠️ Error generating recommendations: {str(e)}")
        print("Showing popular items instead.")
        
        # Fall back to popular items if personalization fails
        item_popularity = np.asarray(train_matrix.sum(axis=0)).flatten()   # Calculate popularity
        popular_indices = np.argsort(-item_popularity)                     # Sort by popularity
        
        # Filter to ensure indices are within bounds
        valid_indices = [i for i in popular_indices if i < len(item_encoder.classes_)][:num_recs]
        
        # Create recommendation dataframe with popular items
        recs_df = pd.DataFrame({
            'StockCode': item_encoder.classes_[valid_indices],             # Map indices to product codes
            'Score': np.linspace(0.5, 0.3, len(valid_indices)),            # Assign declining confidence scores
            'Type': 'Popular Item (Fallback)'                              # Mark as fallback recommendations
        })
    
    # Add customer ID to all recommendations
    recs_df['CustomerID'] = customer_id                                    # Include target customer ID
    
    # Merge with original dataframe to get descriptions
    # Get unique stock codes and descriptions
    stock_desc_df = df[['StockCode', 'Description']].drop_duplicates()     # Create product lookup table
    recs_df = recs_df.merge(stock_desc_df, on='StockCode', how='left')     # Join to get descriptions
    
    # Reorganize columns to display in desired order
    recs_df = recs_df[['CustomerID', 'StockCode', 'Description', 'Type', 'Score']]
    
    return recs_df

# Generate sample recommendations
sample_customer = recommendation_system['user_encoder'].classes_[0]           # Get first customer ID from dataset
print("\nSample recommendations for a customer:" + sample_customer)           # Display customer ID
recommendations = recommend_for_customer(sample_customer, recommendation_system, online_retail_data, num_recs=5)  # Generate recommendations
display(recommendations) 

## 📜 Customer Purchase History Analysis

This cell retrieves and displays the actual purchase history for a specific customer, providing the foundation for comparing and evaluating our recommendation system's suggestions against known preferences.

In [ ]:
# Take Customer ID: 13085 for our analysis from the original dataset
sample_customer_id = 13085                                            # Select a specific customer for analysis
# Let us present the top 10 original purchase history for this customer.
display_original_purchase_history(online_retail_data, sample_customer_id, 10)  # Show customer's actual purchases

## 🎯 Real-Time Product Recommendations

This cell generates personalized product recommendations for customer 13085 in real-time, using our trained recommendation model. The results showcase the model's ability to suggest relevant products based on past purchase patterns.

In [ ]:
# For real-time recommendations for customer 13085
#result = display_inference_result(recommendations, 13085, "realtime", num_results=15)
#display(result)  # Add this line

# Generate sample recommendations
sample_customer = '13085'                                                  # Target customer ID for recommendations
print("\nSample recommendations for a customer:" + sample_customer)        # Display customer identifier
recommendations = recommend_for_customer(sample_customer,                  # Generate personalized recommendations
                                        recommendation_system,             # Using our trained model
                                        online_retail_data,                # And complete dataset for descriptions
                                        num_recs=20)                        # Request 6 recommendations
display(recommendations)  

## 📋 Recommendation Display Utility Function

This utility function creates a beautifully formatted display of product recommendations for a specific customer. It handles column mapping, adds product descriptions, applies visual styling, and works in different inference contexts.

In [ ]:
'''
def display_inference_result(inference_result_df, customer_id, inference_type, num_results=10):
    """
    Display recommendation results in a beautiful table format with customer ID
    
    Args:
        inference_result_df: DataFrame with recommendation results
        customer_id: ID of the customer
        inference_type: Either 'batch' or 'realtime'
        num_results: Number of results to display (default: 10)
    """
    # Make a copy to avoid modifying the original
    result = inference_result_df.copy()                                 # Work with a copy to preserve original data
    
    # Check how many recommendations we actually have
    actual_count = len(result)                                          # Count available recommendations
    if actual_count < num_results:
        print(f"Note: Only {actual_count} recommendations available (requested {num_results})")
    
    # Rename columns to standard names if needed
    column_mapping = {
        "user_id": "CustomerID",                                        # Standardize customer ID column
        "item_id": "StockCode",                                         # Standardize product code column
        "score": "Recommendation Score"                                 # Standardize score column
    }
    
    # Only rename columns that exist
    rename_dict = {}
    for old_col, new_col in column_mapping.items():
        if old_col in result.columns:                                   # Check if column exists before renaming
            rename_dict[old_col] = new_col
    
    if rename_dict:
        result = result.rename(columns=rename_dict)                     # Apply column renaming
    
    # FIXED: Always set CustomerID to the value provided in the parameter
    result["CustomerID"] = customer_id                                  # Ensure customer ID is consistent
    
    # Convert StockCode to string
    if "StockCode" in result.columns:
        result["StockCode"] = result["StockCode"].astype(str)           # Ensure consistent string format for product codes
    
    # Add descriptions using a simpler approach
    if "StockCode" in result.columns:
        try:
            # Create a mapping dictionary for descriptions
            desc_map = dict(zip(
                stock_code_desc_look_up["StockCode"].astype(str),       # Convert keys to strings for matching
                stock_code_desc_look_up["Description"]                  # Use product descriptions as values
            ))
            
            # Add descriptions column
            result["Description"] = result["StockCode"].map(desc_map)   # Map product codes to descriptions
        except Exception as e:
            print(f"Note: Could not add descriptions - {e}")            # Handle errors gracefully
    
    # Reorder columns to ensure CustomerID is first
    cols = ["CustomerID"]                                               # Start with customer ID
    for col in result.columns:
        if col != "CustomerID":
            cols.append(col)                                            # Add all other columns
    result = result[cols]                                               # Reorder columns
    
    # Limit to requested number of results
    result = result.head(num_results)                                   # Trim to requested number
    
    # Print appropriate header
    display_count = min(num_results, actual_count)                      # Use actual count if less than requested
    if inference_type == "batch":
        print(f"\n[ BATCH TRANSFORM ] Top {display_count} Recommendations for Customer ID: {customer_id}")
    else:
        print(f"\n[ REAL-TIME INFERENCE ] Top {display_count} Recommendations for Customer ID: {customer_id}")
    
    # Try to apply styling if in a notebook environment
    try:
        from IPython.display import display                             # Import notebook display functionality
        import pandas as pd                                             # Ensure pandas is available
        
        # Apply styling for a beautiful table
        styled_result = result.style.set_properties(**{
            'background-color': '#f5f5f5',                              # Light gray background
            'border-color': '#888888',                                  # Medium gray border
            'border-style': 'solid',                                    # Solid border style
            'border-width': '1px',                                      # 1px border width
            'border-collapse': 'collapse',                              # Collapse borders
            'padding': '8px',                                           # Comfortable padding
            'font-size': '11pt'                                         # Readable font size
        })
        
        # Format recommendation scores
        if "Recommendation Score" in result.columns:
            styled_result = styled_result.format({
                'Recommendation Score': '{:.4f}'                         # Format scores to 4 decimal places
            })
            
            # Add color gradient to recommendation scores
            styled_result = styled_result.background_gradient(
                cmap='Blues',                                           # Blue color gradient
                subset=['Recommendation Score']                          # Apply only to score column
            )
        
        # Hide index
        try:
            styled_result = styled_result.hide_index()                  # Hide index numbers for cleaner display
        except:
            pass                                                        # Gracefully handle older pandas versions
        
        # Display the styled table
        display(styled_result)                                          # Show the formatted table
        return result
    
    except ImportError:
        # Fall back to regular display if not in a notebook
        print("\nRecommendation Results:")
        print(result.to_string(index=False))                            # Plain text fallback
        return result

'''